In [ ]:
#imports
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend

# Conectando com as pastas do Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Criando as métricas de recall, precisão e f1 medida

In [6]:
# criando métrica recall
def recall_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + backend.epsilon())
    return recall

In [7]:
# criando métrica precisão
def precision_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = backend.sum(backend.round(backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + backend.epsilon())
    return precision

In [8]:
# criando métrica f1
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+backend.epsilon()))

# Desenvolvendo a rede neural 

In [ ]:
# Inicializando a Rede Neural Convolucional
classifier = Sequential()

In [ ]:
# Passo 1 - Primeira Camada de Convolução
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

In [ ]:
# Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

In [ ]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Passo 3 - Flattening
classifier.add(Flatten())

In [ ]:
# Passo 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
# Compilando a rede
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy',f1_m,precision_m, recall_m])

In [ ]:
# Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# verificando as pastas do sistema
import shutil

import os

caminhos = [os.path.join('TCC/dataset_teste/', nome) for nome in os.listdir('TCC/dataset_teste/')]

# retirando as pastas que não serão utilizadas
import shutil
shutil.rmtree('TCC/dataset_teste/.ipynb_checkpoints')

In [ ]:
# Pré-processamento das imagens de treino e validação
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/TCC/dataset_treino',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_set = validation_datagen.flow_from_directory('/content/drive/My Drive/TCC/dataset_teste',
                                                        target_size = (64, 64),
                                                        batch_size = 32,
                                                        class_mode = 'binary')

Found 15163 images belonging to 2 classes.
Found 6496 images belonging to 2 classes.


# Definindo o pesos para as classes benigna e maligna

In [ ]:
class_weight = {0: 1,
                1: 9}

# Executando o treinamento das imagens e salvando o modelo treinado

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

classifier.fit(training_set,
                         epochs = 5,            
                         validation_data = validation_set,
                         class_weight = class_weight
                        )

classifier.save('/content/drive/My Drive/TCC/modelo3649aws.h5')

# Carregando o modelo treinado

In [2]:
import keras
import keras.utils
from keras import utils as np_utils

In [9]:
classifier = keras.models.load_model('/content/drive/My Drive/TCC/modelo3649aws.h5', custom_objects={'f1_m': f1_m, 'recall_m': recall_m, 'precision_m': precision_m})

# Analisando a rede criada

In [10]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

# Verificando a quantidade de acertos das classes benigna e maligna

In [11]:
from keras.preprocessing import image
from matplotlib import pyplot
import numpy as np
from PIL import Image
import glob
import pandas as pd

from collections import Counter
from sklearn.datasets import make_classification
#from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where

In [12]:
respostas = []
for img in glob.glob("/content/drive/My Drive/TCC/dataset_teste/maligno/*.jpg"):
    # Primeira Imagem
    test_image = image.load_img(img, target_size = (64, 64))
    #test_image = image.load_img('benigno/ISIC_0000082.jpg', target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    #training_set.class_indices
    if result[0][0] == 0:
        prediction = 0
    else:
        prediction = 1
    respostas.append(prediction)
len(respostas)

685

In [13]:
sum(respostas)/len(respostas) * 100

81.02189781021897

In [14]:
respostas = []
for img in glob.glob("/content/drive/My Drive/TCC/dataset_teste/benigno/*.jpg"):
    # Primeira Imagem
    test_image = image.load_img(img, target_size = (64, 64))
    #test_image = image.load_img('benigno/ISIC_0000082.jpg', target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    #training_set.class_indices
    if result[0][0] == 0:
        prediction = 1
    else:
        prediction = 0
    respostas.append(prediction)
len(respostas)

5811

In [15]:
sum(respostas)/len(respostas) * 100

86.852521080709